# 训练Xception模型，并保存权重

1 使用 Xception

2 冻结训练分类器

3 finetune训练整个网络

### 1 import libs

In [1]:
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options = tf.GPUOptions(allow_growth=True)
set_session(tf.Session(config=config))

### 2 build pretrained model

In [2]:
with tf.device('/cpu:0'):
    input_tensor = Input((299, 299, 3))
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
    base_model.summary()
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(16, activation='softmax')(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)
    model.summary()

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(optimizer='Adadelta', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
parallel_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 16)           20894264    lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
__________

### 3 build param

In [3]:
batch_size = 64
epochs = 2

total_imgs_num = 152957 + 6130
train_imgs_num = 152957
valid_imgs_num = 8498

img_size = (299, 299)

### 4 build data & train

In [4]:
img_gen_t = ImageDataGenerator()
train_generator = img_gen_t.flow_from_directory("/usr/super-speed-data/train-data-xception/20181029/train-ready/", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/usr/super-speed-data/train-data-xception/20181029/valid-ready/",
                                                img_size, shuffle=True, batch_size=batch_size)


parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=train_imgs_num//batch_size, 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=valid_imgs_num//batch_size)

model.save_weights("Xception_first_train.h5")

Found 152957 images belonging to 16 classes.
Found 8498 images belonging to 16 classes.
Epoch 1/2
2389/2389 [==============================] - 474s 198ms/step - loss: 0.7941 - acc: 0.7460 - val_loss: 1.2288 - val_acc: 0.6102
Epoch 2/2
2389/2389 [==============================] - 476s 199ms/step - loss: 0.6815 - acc: 0.7725 - val_loss: 1.1997 - val_acc: 0.6141


### 5 Finetune

In [5]:
with tf.device('/cpu:0'):
    input_tensor = Input((299, 299, 3))
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(16, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('Xception_first_train.h5')

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(optimizer='Adadelta', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_4[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [6]:
batch_size = 48
epochs = 50

total_imgs_num = 152957 + 8498
train_imgs_num = 152957
valid_imgs_num = 8498

img_size = (299, 299)

img_gen_t = ImageDataGenerator(rotation_range= 90,                            
                               width_shift_range= 0.1,
                               height_shift_range= 0.1,
                               zoom_range= 0.1,
                               brightness_range=[0.9, 1.1],
#                               zca_whitening= True,
                               horizontal_flip= True,
                               vertical_flip= True)

train_generator = img_gen_t.flow_from_directory("/usr/super-speed-data/train-data-xception/20181029/train-ready/", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/usr/super-speed-data/train-data-xception/20181029/valid-ready/", 
                                                img_size, shuffle=True, batch_size=batch_size)


parallel_model.fit_generator(generator=train_generator, steps_per_epoch=train_imgs_num//batch_size, epochs=epochs, verbose=1,
                   validation_data=valid_generator, validation_steps=valid_imgs_num//batch_size)

gen = ImageDataGenerator()

model.save_weights("Xception_finetune_50.h5")

Found 152957 images belonging to 16 classes.
Found 8498 images belonging to 16 classes.
Epoch 1/50
3186/3186 [==============================] - 2400s 753ms/step - loss: 0.3761 - acc: 0.8698 - val_loss: 0.2920 - val_acc: 0.9017
Epoch 2/50
3186/3186 [==============================] - 2391s 751ms/step - loss: 0.2773 - acc: 0.9014 - val_loss: 0.2567 - val_acc: 0.9055
Epoch 3/50
3186/3186 [==============================] - 2388s 749ms/step - loss: 0.2493 - acc: 0.9098 - val_loss: 0.2250 - val_acc: 0.9200
Epoch 4/50
3186/3186 [==============================] - 2399s 753ms/step - loss: 0.2303 - acc: 0.9160 - val_loss: 0.2385 - val_acc: 0.9155
Epoch 5/50
3186/3186 [==============================] - 2382s 748ms/step - loss: 0.2198 - acc: 0.9196 - val_loss: 0.2397 - val_acc: 0.9117
Epoch 6/50
3186/3186 [==============================] - 2390s 750ms/step - loss: 0.2103 - acc: 0.9233 - val_loss: 0.2424 - val_acc: 0.9081
Epoch 7/50
3186/3186 [==============================] - 2388s 749ms/step - los

In [22]:
batch_size = 128
img_size = (299, 299)

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/usr/super-speed-data/train-data-xception/20181019/test-ready/", 
                                         img_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
test = parallel_model.predict_generator(test_generator, (test_img_nums//batch_size) + 1, verbose=1)
test = test[:test_img_nums]
label = test_generator.classes

Found 6131 images belonging to 16 classes.
48/48 [==============================] - 20s 411ms/step


In [23]:
print(test.shape)
print(label.shape)

(6131, 16)
(6131,)


In [24]:
print(len(test))
print(len(label))

print(test[0])
print(np.argmax(test[0]))

true_nums = 0
total_nums = len(label)
for i in range(len(label)):
    if(label[i] == np.argmax(test[i])):
        true_nums = true_nums + 1

print(true_nums)
print(total_nums)
print(true_nums / total_nums)

6131
6131
[1.0000000e+00 2.1713600e-35 0.0000000e+00 1.5930831e-38 3.9584648e-19
 8.5789389e-24 1.3994146e-26 0.0000000e+00 0.0000000e+00 0.0000000e+00
 3.8429720e-35 0.0000000e+00 0.0000000e+00 4.7731313e-32 1.1182708e-27
 2.0915513e-25]
0
5876
6131
0.9584080900342522


### 5 load weights & predict

In [43]:
input_tensor = Input((299, 299, 3))
x = Lambda(xception.preprocess_input)(input_tensor)

base_model = Xception(input_tensor=x, weights=None, include_top=False)
m_out = base_model.output
p_out = GlobalAveragePooling2D()(m_out)
p_out = Dropout(1.0)(p_out)
fc_out = Dense(1024, activation='relu')(p_out)
fc_out = Dropout(1.0)(fc_out)
predictions = Dense(3, activation='softmax')(fc_out)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

In [44]:
img_gen_s = ImageDataGenerator()
submit_generator = img_gen_t.flow_from_directory("../dataset/test", img_size, shuffle=False, batch_size=2)

predictions = []
for i, batch_data in enumerate(submit_generator):
    if (i > 1):
        break
    batch_img_data = np.array(batch_data[0])
    predictions.extend(model.predict_on_batch(batch_img_data))
    print(i)
    
print(predictions)

Found 14 images belonging to 3 classes.
0
1
[array([ 0.30176267,  0.35524741,  0.34298989], dtype=float32), array([ 0.28475785,  0.34978592,  0.36545616], dtype=float32), array([ 0.3018229 ,  0.35525236,  0.34292471], dtype=float32), array([ 0.29986629,  0.35433695,  0.34579682], dtype=float32)]


In [45]:
print(np.array(predictions))

[[ 0.30176267  0.35524741  0.34298989]
 [ 0.28475785  0.34978592  0.36545616]
 [ 0.3018229   0.35525236  0.34292471]
 [ 0.29986629  0.35433695  0.34579682]]


### 6 write predictions 2 CSV